In [1]:
print(123)

123


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f



### 注意观察下面的代码，你的输入需要按照batch channel w h 来设置。
- pandding=0 不是说不补，而是说你的输入输出是一致的，而当你设置padding=1的时候，那就意味着你的输出比你的输入大一圈。
- 可以通过实验感受下。

In [26]:
t1 = torch.rand(1,3,4,4)
conv1 = torch.nn.Conv2d(3,2,1,1,1)
conv1(t1),conv1.bias,conv1.weight
None

### 接下来我们感受下什么是BN
- BN操作针对的是通道这个要注意，所以有个参数要设置对。
- 直接求var的时候 要考虑有偏和无偏，什么是var的有偏估计呢？有偏分母就是n-1

In [73]:
t2 = torch.arange(1,25,dtype=torch.float32)
t3 = t2.reshape(1,2,3,4)
t3

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.]],

         [[13., 14., 15., 16.],
          [17., 18., 19., 20.],
          [21., 22., 23., 24.]]]])

In [86]:
bn = nn.BatchNorm2d(2)
bn(t3)


tensor([[[[-1.5933, -1.3036, -1.0139, -0.7242],
          [-0.4345, -0.1448,  0.1448,  0.4345],
          [ 0.7242,  1.0139,  1.3036,  1.5933]],

         [[-1.5933, -1.3036, -1.0139, -0.7242],
          [-0.4345, -0.1448,  0.1448,  0.4345],
          [ 0.7242,  1.0139,  1.3036,  1.5933]]]],
       grad_fn=<NativeBatchNormBackward0>)

In [87]:
bn.running_mean, bn.running_var,bn.weight,bn.bias
None

In [111]:
t = torch.arange(13,25,dtype=torch.float32)

sum((t-18.5)**2) / 12

tensor(11.9167)

In [104]:
(t3-18.5)/(11.9167**0.5+0.00001)

tensor([[[[-5.0694, -4.7797, -4.4901, -4.2004],
          [-3.9107, -3.6210, -3.3313, -3.0417],
          [-2.7520, -2.4623, -2.1726, -1.8829]],

         [[-1.5932, -1.3036, -1.0139, -0.7242],
          [-0.4345, -0.1448,  0.1448,  0.4345],
          [ 0.7242,  1.0139,  1.3036,  1.5932]]]])

### spp中的pool操作

In [35]:
x = torch.rand(3,64,16,16)
x1 = f.max_pool2d(x,9,stride=1,padding=4)
x2 = f.max_pool2d(x,5,stride=1,padding=2)
torch.cat([x1,x2],dim=1).shape

torch.Size([3, 128, 16, 16])

### 看下什么是残差块

In [37]:

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        print(identity.shape)

        out = self.conv1(x)
        print(out.shape)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        print(out.shape)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
    
model = BasicBlock(64,64)
model(x).shape


torch.Size([3, 64, 16, 16])
torch.Size([3, 64, 16, 16])
torch.Size([3, 64, 16, 16])


torch.Size([3, 64, 16, 16])

In [20]:
forward(nn, x)

AttributeError: module 'torch.nn' has no attribute 'conv1'

In [ ]:
你好啊
这是个一个test
